# 自然语言识别处理单元
## [拂晓工作室](https://github.com/Errrneist/Alchemist)
* 此程序可用于识别文字中的语言以及深度分析新闻概况

# 参考资料
* [1] [SFrame入门](https://apple.github.io/turicreate/docs/api/generated/turicreate.SFrame.html#turicreate.SFrame)
* [2] [从字符串中删除子字符串](https://stackoverflow.com/questions/31273642/better-way-to-remove-multiple-words-from-a-string)
* [3] [用Python分析《红楼梦》 - 楼宇](https://zhuanlan.zhihu.com/p/29209681)
* [4] [使用pyltp包进行中文分词实例](https://blog.csdn.net/sinat_26917383/article/details/77067515)
* [5] [pyltp技术文档](http://pyltp.readthedocs.io/zh_CN/latest/api.html#id15)
* [6] [pyltp介绍文档一](https://www.ltp-cloud.com/intro/#introduction )
* [7] [pyltp介绍文档二](http://ltp.readthedocs.io/zh_CN/latest/appendix.html#id5)
* [8] [pyltp深度训练模型](https://pan.baidu.com/share/link?shareid=1988562907&uk=2738088569#list/path=/)


# 导入库

In [18]:
# 导入库
import urllib
import re
import pymysql
import os
import csv
import time
import random
import datetime
# import pyltp
import turicreate as tc
from bs4 import BeautifulSoup

# 载入数据

In [74]:
news = tc.SFrame('../DataSets/Eastmoney/News/China/CHINA20180701-0848.csv')
news

Finished parsing file /Users/hongjunwu/Documents/GitHub/Alchemist/DataSets/Eastmoney/News/China/CHINA20180701-0848.csv

Parsing completed. Parsed 100 lines in 0.053261 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,int,list,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/hongjunwu/Documents/GitHub/Alchemist/DataSets/Eastmoney/News/China/CHINA20180701-0848.csv

Parsing completed. Parsed 515 lines in 0.039952 secs.

contents,date,day,month,related,title
据中国江西网报道，6月27日，南昌市公安局红谷滩分局透露，为进一步深化南昌市户籍制度改革工作 ...,20180627,27,6,[],江西南昌放开非户籍人口落户领取居住证满半年即可申请 ...
足球场上的大热必死，用经济学的语言说是冲高回落。虽然经济学家们深谙风险对冲之道，但在看球时， ...,20180630,30,6,"[高盛, 银行]",投行眼里的世界杯
据中国人民银行网站消息，中国人民银行货币政策委员会2018年第二季度例会于6月27日在北京召 ...,20180628,28,6,[],央行：把握好结构性去杠杆的力度和节奏
北京6月27日讯今日，中国政府网官方微博发布博文“快快快看！这里有份国务院政策利好，用一个字 ...,20180627,27,6,[微博],“中小微企业日” 闪送却调皮了一下
中国人民银行行长易纲29日表示，小微企业金融服务需要全社会共同努力，切实把货币政策、差异化监 ...,20180630,30,6,[银行],央行行长：小微企业金融服务需要全社会共同努力
2018年6月30日国家统计局服务业调查中心和中国物流与采购联合会发布了中国采购经理指数。对 ...,20180630,30,6,[银行],统计局解读：非制造业商务活动指数持续上升
6月30日，国家发展改革委、商务部以第19号令，发布了《自由贸易试验区外商投资准入特别管理措 ...,20180630,30,6,[],2018年版自贸试验区外资准入负面清单出炉！
6月29日，中国人民银行办公厅发布关于支付机构客户备付金全部集中交存有关事宜的通知。通知明确 ...,20180630,30,6,[银行],央行：将按月提高支付机构客户备付金集中交存比例
6月26日，东丽区召开环境保护专题会议，研究进一步改善东丽区生态环境，加强环境保护的工作举措 ...,20180628,28,6,[强生],天津东丽区加强环境保护 改善生态环境
外管局：3月末中国银行业对外金融净负债2817亿美元，其中，人民币净负债2841亿美元，外币 ...,20180627,27,6,[],外管局：3月末中国银行业对外金融净负债2817亿美元 ...


# 检查数据纯净度

In [75]:
# 随机人工检查数据纯净度

def showContents(number_to_show, total_amount):
    counter = 0
    while counter < number_to_show:
        print(news['contents'][int(random.uniform(0, total_amount - 1))])
        print('')
        counter += 1
    
showContents(3, len(news['contents']))

据国家统计局30日消息，2018年6月份，中国制造业采购经理指数(PMI)为51.5%，比上月回落0.4个百分点，仍高于上半年均值0.2个百分点，制造业总体继续保持扩张态势。从企业规模看，大型企业PMI为52.9%，比上月回落0.2个百分点，保持在临界点之上；中型企业PMI为49.9%，比上月下降1.1个百分点，落至临界点以下；小型企业PMI为49.8%，比上月回升0.2个百分点，仍低于临界点。从分类指数看，在构成制造业PMI的5个分类指数中，生产指数、新订单指数和供应商配送时间指数高于临界点，原材料库存指数和从业人员指数低于临界点。生产指数为53.6%，低于上月0.5个百分点，仍处于扩张区间，表明制造业生产保持增长，增速有所放缓。新订单指数为53.2%，比上月回落0.6个百分点，继续高于临界点，表明制造业市场需求保持扩张，增幅有所收窄。原材料库存指数为48.8%，比上月下降0.8个百分点，位于临界点以下，表明制造业主要原材料库存量继续减少。从业人员指数为49.0%，微低于上月0.1个百分点，位于临界点以下，表明制造业企业用工量收缩。供应商配送时间指数为50.2%，高于上月0.1个百分点，位于临界点之上，表明制造业原材料供应商交货时间有所加快。

北京市16个区与内蒙古自治区31个国贫旗县日前分别签订了《京蒙劳务协作扶贫行动协议》。按照协议，北京16个区将帮助内蒙古自治区国贫旗县有转移意愿的劳动力实现转移就业，并协助内蒙古输出人员开展职业培训，提供职业指导帮助，推动内蒙古脱贫攻坚。内蒙古自治区人社厅相关负责人告诉中国经济时报记者，京蒙劳务协作专项扶贫行动是促进内蒙古深入实施京蒙对口帮扶、开展人力资源社会保障脱贫攻坚的重要举措。根据协议内容，内蒙古签约地区负责摸清本地区现有转移就业意愿和能力的劳动力，特别是建档立卡贫困劳动力的基本情况、就业意愿及培训需求，形成“求职需求清单”。北京签约方将根据“求职需求清单”，组织发动管理规范、岗位适合、待遇较好、社会责任感较强的企业定向筛选和开发岗位，形成“岗位供给清单”。最终签约双方建立区域劳动力供求信息收集、分析、评估和定期发布制度，通过举办劳动力信息推介会和用人单位招聘会等形式，实现人力资源与岗位需求有效对接。

近年来，青岛开发区紧紧围绕推动转型升级和创新发展主线，创新外资利用方式，推动开发区工作再上新台阶。在商务部发布的2

# 自然语言识别